In [ ]:
from ellipse_rcnn.utils.conics import (
    ellipse_to_conic_matrix,
    bbox_ellipse,
)
import torch

from ellipse_rcnn.utils.data.fddb import FDDB

In [ ]:
semimajor_axis, semiminor_axis = torch.tensor([5.0, 6.0]), torch.tensor([3.0, 4.0])
cx, cy = torch.tensor([1.0, 2.0]), torch.tensor([3.0, 4.0])
theta = torch.tensor([0.0, 0.0])

ellipse_matrices = ellipse_to_conic_matrix(
    semimajor_axis, semiminor_axis, cx, cy, theta
)

expected_bbox = torch.tensor([[-4, 0.0, 6.0, 6.0], [-4.0, 0.0, 8.0, 8.0]])
calculated_bbox = bbox_ellipse(ellipse_matrices)

# assert torch.allclose(calculated_bbox, expected_bbox), "Bounding box calculation #1 failed"
expected_bbox, calculated_bbox

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Rectangle, Ellipse


def plot_bboxes_ellipses(_bboxes: np.ndarray, _theta: np.ndarray):
    """
    Plot the bounding boxes and ellipses on the image.

    Parameters
    ----------
    _bboxes
    _ellipses
    Returns
    -------

    """
    fig, ax = plt.subplots(1, figsize=(15, 25))
    ax.set_ylim(-10, 10)
    ax.set_xlim(-10, 10)
    ax.set_aspect("equal")

    for (x_min, y_min, x_max, y_max), t in zip(_bboxes, _theta):
        cx, cy, w, h = (
            x_min + (x_max - x_min) / 2,
            y_min + (y_max - y_min) / 2,
            x_max - x_min,
            y_max - y_min,
        )
        rect = Rectangle((x_min, y_min), w, h, fill=False, color="r")
        ax.add_patch(rect)

        ellipse = Ellipse((cx, cy), w, h, angle=t, alpha=0.3)

        ax.add_patch(ellipse)

In [ ]:
plot_bboxes_ellipses(calculated_bbox, theta)

In [ ]:
from ellipse_rcnn.utils.viz import DetectionPlotter

In [ ]:
from ellipse_rcnn import EllipseRCNN
from ellipse_rcnn.core.model import EllipseRCNNLightning
import torch

In [ ]:
model = EllipseRCNN()
pl_model = EllipseRCNNLightning(model=model)

In [ ]:
with open("../checkpoints/ellipse_rcnn_model.pth", "rb") as f:
    pl_model.load_state_dict(torch.load(f, weights_only=True))

In [ ]:
model.eval()

In [ ]:
ds = FDDB("../data/FDDB")

In [ ]:
from PIL import Image
import numpy as np

image, target_dict = ds[5]

image_reshaped = np.transpose(
    image.numpy(), (1, 2, 0)
)  # Moves the channel to the last axis

if image_reshaped.dtype != np.uint8:
    # Assuming the array is normalized to [0, 1], scale to [0, 255]
    image_reshaped = image_reshaped.astype(np.uint8)

# Step 3: Convert to Pillow image
img = Image.fromarray(image_reshaped, mode="RGB")

# Save or display the image
pred = model(image.unsqueeze(0))
if len(pred[0]["boxes"]) > 0:
    plotter = DetectionPlotter.from_ellipses(pred[0]["ellipse_matrices"].detach())
    fig, ax = plt.subplots(1, figsize=(15, 25))
    ax.set_aspect("equal")
    ax.grid(True)
    # ax.set_xlim(-10, 10)
    # ax.set_ylim(-10, 10)
    ax.imshow(img)
    plotter.plot(ax)
    plt.show()

In [ ]:
from matplotlib import pyplot as plt

# Create a figure and an axis
fig, ax = plt.subplots(figsize=(10, 8))
ax.imshow(img)
ax.axis("off")  # Turn off the axis for better visualization
plt.show()  # Display the plot

In [ ]:
plotter = DetectionPlotter()